In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from python.fluorophores import FlStatic
from python.psfs import PsfVectorial
from python.estimators import est_qLSQiter2D
from python.simulators import Simulator

In [ ]:
psf_vecph = PsfVectorial()
pinholepos = np.arange(0, 500, 100)  # um

stdx = np.zeros((len(pinholepos),3))
crb1 = np.zeros((len(pinholepos),3))
biasx = np.zeros((len(pinholepos),3))
rmsex = np.zeros((len(pinholepos),3))
phot = np.zeros((len(pinholepos)))

In [ ]:
fl = FlStatic(brightness=1000)  # define a static fluorophore
fl.pos = [0, 0, 0]

sim = Simulator(fluorophores=fl)

numberOfLocalizations=1000

# define scan pattern
L = 75
zeroposx = np.atleast_2d(np.array([-1,1,0])*L/2)
probecenter = True  # should we also probe the center?
orbitpoints = 6
laserpower = 5  # relative, increases brightness
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 1  # how often to repeat the pattern scan

sim.defineComponent("estdonut", "estimator", est_qLSQiter2D, parameters=[L, probecenter], dim=(0,1))

In [ ]:
psfall = []
for k in range(len(pinholepos)):
    psf_vecph.setpinhole(AU=1, offset=[pinholepos[k], 0])
    sim.definePattern("ph_misaligned", 
                      psf_vecph, 
                      phasemask="vortex", 
                      makepattern="orbitscan", 
                      orbitpoints=orbitpoints,
                      probecenter=probecenter,
                      orbitL=L,
                      pointdwelltime=0.1,
                      laserpower=laserpower,
                      repetitions=repetitions)
    stack, gridv = psf_vecph.imagestack("vortex")
    psfall.append(stack)
    seq = ["ph_misaligned", "estdonut"]
    out=sim.runSequence(seq, maxlocs=10000)
    sr=sim.summarize_results(out)
    stdx[k,:] = sr.std
    crb1[k,:] = sr.sCRB1
    biasx[k,:] = sr.bias
    rmsex[k,:] = sr.rmse
    phot[k] = sr.phot

zind=2
stdxrel=stdx/crb1[0,:]*np.sqrt(phot)[...,None]  # normalized to perfectly aligned phaseplate and photon numbers
rmserel=rmsex/crb1[0,:]*np.sqrt(phot)[...,None]  # normalized to perfectly aligned phaseplate and photon numbers
crbrel = crb1/crb1[0,:]
crb = crb1/np.sqrt(phot)[...,None]

psfall = np.stack(psfall,axis=3)  # to array

In [ ]:
fig = plt.figure(figsize=(10,5))
gs = GridSpec(2, 3, figure=fig)

ax = fig.add_subplot(gs[0,0])
ax.plot(pinholepos,stdxrel[:,0].T,'k')
ax.plot(pinholepos,stdxrel[:,1].T,'k--')
ax.set_xlabel('misalignment of pinhole (nm)')
ax.set_ylabel('std / CRB aligned')
ax.set_title("Standard deviation")
ax.legend(["x","y"])
ax.set_ylim([0.9,1.1])

ax = fig.add_subplot(gs[0,2])
ax.plot(pinholepos,biasx[:,0],'k')
ax.plot(pinholepos,biasx[:,1],'k--')
ax.set_xlabel('misalignment of pinhole (nm)')
ax.set_ylabel('bias (nm)')
ax.set_title("Bias")
ax.set_ylim([-1,1])

ax = fig.add_subplot(gs[0,1])
ax.plot(pinholepos,rmserel[:,0].T,'k') 
ax.plot(pinholepos,rmserel[:,1].T,'k--')
ax.set_xlabel('misalignment of pinhole (nm)')
ax.set_ylabel('rmse / CRB aligned')
ax.set_title("Root mean square error (rmse)")
ax.set_ylim([0.9,1.1])


In [ ]:
indz0 = np.where(gridv[2] >= 0)[0][0].astype(int)
for k in range(len(pinholepos)):
    if k==0:
        psfx = psfall[:,:,indz0,k]
    else:
        psfx = np.hstack([psfx,psfall[:,:,indz0,k]])
ax = fig.add_subplot(gs[1,:])
ax.imshow(psfx)
ax.axis('off')
ax.set_title('misalignment')


In [ ]:
fig.tight_layout()